In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# 1. Load Data
# ------------
try:
    attack_df = pd.read_csv('final_attack_profiles_long (3).csv')
    target_df = pd.read_csv('final_target_profiles_long (3).csv')
    members_df = pd.read_csv('cluster_members_data.csv')
except FileNotFoundError:
    print("Please ensure the CSV files are in the same directory.")

# 2. Setup Colors and Categories
# ------------------------------
clusters = sorted(attack_df['Cluster'].unique())
colors = px.colors.qualitative.Bold
cluster_color_map = {c: colors[i % len(colors)] for i, c in enumerate(clusters)}

# 3. Helper Functions
# -------------------
def get_radar_data(df, category_col, value_col, cluster):
    subset = df[df['Cluster'] == cluster].sort_values(by=category_col)
    r = subset[value_col].tolist()
    theta = subset[category_col].tolist()
    if r:
        r.append(r[0])
        theta.append(theta[0])
    return r, theta

def get_table_data(df, cluster=None):
    if cluster:
        subset = df[df['Cluster'] == cluster]
    else:
        subset = df
    display_cols = ['GroupName', 'Country']
    return [subset[col] for col in display_cols]

def hex_to_rgba(hex_color, opacity=0.2):
    hex_color = hex_color.lstrip('#')
    rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    return f"rgba({rgb[0]}, {rgb[1]}, {rgb[2]}, {opacity})"

# 4. Create Figure with Subplots
# ------------------------------
fig = make_subplots(
    rows=1, cols=3,
    column_widths=[0.32, 0.32, 0.36],
    specs=[[{'type': 'polar'}, {'type': 'polar'}, {'type': 'table'}]],
    subplot_titles=("<b>Attack Profile</b>", "<b>Target Profile</b>", "<b>Cluster Members</b>"),
    horizontal_spacing=0.08
)

# 5. Add Traces
# -------------
traces_per_cluster = 3

for cluster in clusters:
    color = cluster_color_map[cluster]

    if color.startswith('#'):
        rgba_fillcolor = hex_to_rgba(color, 0.2)
    else:
        rgba_fillcolor = color

    # -- A. Attack Radar (Col 1) --
    r_att, theta_att = get_radar_data(attack_df, 'Attack Type', 'Attack Type Percentage', cluster)
    fig.add_trace(go.Scatterpolar(
        r=r_att, theta=theta_att,
        fill='toself',
        fillcolor=rgba_fillcolor,
        line=dict(color=color, width=2),
        marker=dict(size=4),
        name=f'Cluster {cluster}',
        legendgroup=f'Cluster {cluster}',
        hovertemplate=f"<b>Cluster {cluster}</b><br>%{{theta}}: %{{r:.1f}}%<extra></extra>"
    ), row=1, col=1)

    # -- B. Target Radar (Col 2) --
    r_tar, theta_tar = get_radar_data(target_df, 'Target Type', 'Target Type Percentage', cluster)
    fig.add_trace(go.Scatterpolar(
        r=r_tar, theta=theta_tar,
        fill='toself',
        fillcolor=rgba_fillcolor,
        line=dict(color=color, width=2),
        marker=dict(size=4),
        name=f'Cluster {cluster}',
        legendgroup=f'Cluster {cluster}',
        showlegend=False,
        hovertemplate=f"<b>Cluster {cluster}</b><br>%{{theta}}: %{{r:.1f}}%<extra></extra>"
    ), row=1, col=2)

    # -- C. Member Table (Col 3) --
    header_values = ["<b>Group Name</b>", "<b>Country</b>"]
    cell_values = get_table_data(members_df, cluster)

    fig.add_trace(go.Table(
        header=dict(values=header_values, fill_color=color, align='left', font=dict(color='white', size=11)),
        cells=dict(values=cell_values, fill_color='#F5F8FF', align='left', font=dict(color='black', size=10), height=25),
        domain=dict(x=[0.7, 1], y=[0, 1])
    ), row=1, col=3)

# -- D. "Master" Table --
all_cell_values = get_table_data(members_df, cluster=None)
fig.add_trace(go.Table(
    header=dict(values=["<b>Group Name</b>", "<b>Country</b>"], fill_color='#333333', align='left', font=dict(color='white', size=11)),
    cells=dict(values=all_cell_values, fill_color='#F5F8FF', align='left', font=dict(color='black', size=10), height=25)
), row=1, col=3)

# 6. Create Dropdown Menus
# ------------------------
buttons = []
total_clusters = len(clusters)
total_traces = len(fig.data)
master_table_index = total_traces - 1

# Option: ALL Clusters
visible_all = [False] * total_traces
for i in range(total_clusters):
    base = i * traces_per_cluster
    visible_all[base] = True
    visible_all[base+1] = True
    visible_all[base+2] = False
visible_all[master_table_index] = True

buttons.append(dict(
    label="All Clusters",
    method="update",
    args=[{"visible": visible_all},
          {"title": "Analysis: All Clusters"}]
))

# Option: Individual Clusters
for i, cluster in enumerate(clusters):
    visible = [False] * total_traces
    base = i * traces_per_cluster

    visible[base] = True
    visible[base+1] = True
    visible[base+2] = True

    buttons.append(dict(
        label=f"Cluster {cluster}",
        method="update",
        args=[{"visible": visible},
              {"title": f"Analysis: Cluster {cluster}"}]
))

# 7. Layout Adjustments
# ---------------------
fig.update_layout(
    title_text="Analysis: All Clusters",
    title_x=0.5,
    title_y=0.98,

    # DROPDOWN
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        direction="down",
        x=1.0,
        xanchor="right",
        y=1.15,
        yanchor="top",
        bgcolor='white', bordercolor='gray', borderwidth=1
    )],

    legend=dict(
        orientation="h",
        yanchor="bottom", y=-0.1,
        xanchor="center", x=0.5
    ),
    font=dict(family="Arial, sans-serif", size=12),
    height=600,

    # --- POLAR CHART FIXES ---
    polar=dict(
        radialaxis=dict(
            visible=True,
            showticklabels=False,
            ticks='',
            layer='below traces'
        ),
        domain = dict(x=[0, 0.32])
    ),
    polar2=dict(
        radialaxis=dict(
            visible=True,
            showticklabels=False,
            ticks='',
            layer='below traces'
        ),
        domain = dict(x=[0.35, 0.67])
    ),

    margin=dict(t=120, b=50, l=40, r=40),
    paper_bgcolor='white',
    plot_bgcolor='white'
)

fig.show()